# RAG Chatbot with Local HuggingFace Model

A complete Retrieval-Augmented Generation (RAG) system for students - **NO API TOKEN NEEDED!**

## What This Notebook Does

1. **Loads PDF documents** from the `documents/` folder
2. **Chunks text** into smaller pieces with overlap
3. **Creates embeddings** (vector representations) locally
4. **Builds a search index** using FAISS
5. **Retrieves relevant chunks** for your questions
6. **Generates answers** using a local HuggingFace model (runs on your computer)

## Requirements

- **NO API token needed** - everything runs locally!
- PDF files in the `documents/` folder
- Required packages: `pypdf`, `sentence-transformers`, `faiss-cpu`, `transformers`, `gradio`

## Step 1: Configuration (Global Variables)

**IMPORTANT**: This cell configures the local model - no API token needed!

In [ ]:
# ============================================
# GLOBAL CONFIGURATION
# ============================================

# Local Model Configuration (No API token needed!)
MODEL_NAME = "google/flan-t5-large"  # You can also use "google/flan-t5-base" for faster, smaller model

# Folder containing your PDF documents
PDF_FOLDER = "documents"

print("✅ Configuration loaded!")
print(f"   Local Model: {MODEL_NAME}")
print(f"   PDF Folder: {PDF_FOLDER}")
print(f"   🎉 No API token needed - everything runs locally!")

## Step 2: Import Required Libraries

In [ ]:
import os
from typing import List, Dict
from pathlib import Path

# PDF processing
from pypdf import PdfReader

# Embeddings
from sentence_transformers import SentenceTransformer

# Vector store
import faiss
import numpy as np

# Local LLM
from transformers import pipeline

# UI (optional)
import gradio as gr

print("✅ All libraries imported successfully!")

## Step 3: Initialize the RAG System

This sets up:
- Local HuggingFace model (downloads once, then cached)
- Local embedding model (runs on your computer)

In [ ]:
print("=" * 60)
print("🚀 INITIALIZING RAG CHATBOT")
print("=" * 60)

# Load embedding model (this runs locally)
print("\n✓ Loading embedding model (local)...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("  ✓ Embedding model loaded")

# Load local LLM for answer generation
print(f"\n✓ Loading language model: {MODEL_NAME}")
print("  (This may take a minute on first run - model will be downloaded and cached)")
llm = pipeline(
    "text2text-generation",
    model=MODEL_NAME,
    max_length=512,
    device=-1  # Use CPU (-1) or GPU (0) if available
)
print("  ✓ Language model loaded and ready!")

# Initialize storage
chunks = []
chunk_embeddings = None
index = None

print("\n" + "=" * 60)
print("✅ RAG CHATBOT READY - 100% LOCAL!")
print("=" * 60)

## Step 4: Load PDF Documents

This function extracts text from all PDFs in your `documents/` folder.

In [4]:
def load_pdfs(pdf_folder: str = PDF_FOLDER) -> List[str]:
    """
    Load and extract text from all PDFs in folder
    
    Args:
        pdf_folder: Path to folder containing PDFs
        
    Returns:
        List of text strings, one per PDF
    """
    print(f"\n📄 Loading PDFs from '{pdf_folder}/'...")

    all_text = []
    pdf_folder_path = Path(pdf_folder)
    pdf_folder_path.mkdir(exist_ok=True)

    pdf_files = list(pdf_folder_path.glob("*.pdf"))

    if not pdf_files:
        print(f"  ⚠️  No PDF files found in '{pdf_folder}/'")
        return []

    print(f"  Found {len(pdf_files)} PDF file(s)")

    for pdf_path in pdf_files:
        print(f"  Processing: {pdf_path.name}")
        try:
            reader = PdfReader(pdf_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            all_text.append(text)
            print(f"    ✓ Extracted {len(reader.pages)} pages")
        except Exception as e:
            print(f"    ✗ Error: {e}")

    return all_text

print("✅ Function defined: load_pdfs()")

✅ Function defined: load_pdfs()


## Step 5: Chunk Text with Overlap

Breaking documents into chunks helps with:
- More precise retrieval
- Staying within model limits
- Better matching with queries

**Overlap** ensures we don't lose context at chunk boundaries.

In [5]:
def chunk_text(texts: List[str], chunk_size: int = 500, overlap: int = 100) -> List[str]:
    """
    Split texts into smaller chunks with overlap
    
    Args:
        texts: List of text strings to chunk
        chunk_size: Target size of each chunk in characters
        overlap: Number of characters to overlap between chunks
        
    Returns:
        List of text chunks
    """
    print(f"\n✂️  Chunking text (size={chunk_size}, overlap={overlap})...")

    chunks = []

    for text in texts:
        sentences = text.replace('\n', ' ').split('. ')

        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            sentence_length = len(sentence) + 2

            if current_length + sentence_length > chunk_size and current_chunk:
                chunk_text = '. '.join(current_chunk) + '.'
                chunks.append(chunk_text)

                if overlap > 0:
                    overlap_text = chunk_text[-overlap:]
                    current_chunk = [overlap_text + sentence]
                    current_length = len('. '.join(current_chunk))
                else:
                    current_chunk = [sentence]
                    current_length = sentence_length
            else:
                current_chunk.append(sentence)
                current_length += sentence_length

        if current_chunk:
            chunk_text = '. '.join(current_chunk)
            if not chunk_text.endswith('.'):
                chunk_text += '.'
            chunks.append(chunk_text)

    print(f"  ✓ Created {len(chunks)} chunks")
    if chunks:
        print(f"  ✓ Avg length: {sum(len(c) for c in chunks) / len(chunks):.0f} chars")

    return chunks

print("✅ Function defined: chunk_text()")

✅ Function defined: chunk_text()


## Step 6: Create Vector Store

This creates:
1. **Embeddings**: Vector representations of text chunks
2. **FAISS Index**: Fast similarity search structure

In [6]:
def create_vector_store(text_chunks: List[str]):
    """
    Create embeddings and FAISS index
    
    Args:
        text_chunks: List of text chunks
        
    Returns:
        Tuple of (embeddings, index)
    """
    global chunk_embeddings, index
    
    print(f"\n🔢 Creating vector store...")

    print(f"  Encoding {len(text_chunks)} chunks...")
    chunk_embeddings = embedder.encode(
        text_chunks,
        show_progress_bar=True,
        batch_size=32
    )

    print(f"  Building FAISS index...")
    dimension = chunk_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(chunk_embeddings.astype('float32'))

    print(f"  ✓ Vector store ready ({index.ntotal} vectors)")
    
    return chunk_embeddings, index

print("✅ Function defined: create_vector_store()")

✅ Function defined: create_vector_store()


## Step 7: Setup Knowledge Base

**Run this cell to process your PDFs!**

This will:
1. Load all PDFs from `documents/` folder
2. Chunk the text
3. Create embeddings and search index

In [7]:
# Load PDFs
documents = load_pdfs(PDF_FOLDER)

if not documents:
    print("\n❌ No PDFs loaded. Please add PDFs to the 'documents/' folder.")
else:
    # Chunk texts
    chunks = chunk_text(documents)

    # Create vector store
    chunk_embeddings, index = create_vector_store(chunks)

    print("\n" + "=" * 60)
    print("✅ KNOWLEDGE BASE READY")
    print(f"   Total chunks: {len(chunks)}")
    print(f"   Embedding dim: {chunk_embeddings.shape[1]}")
    print("=" * 60)


📄 Loading PDFs from 'documents/'...
  Found 1 PDF file(s)
  Processing: intro-to-econometrics.pdf
    ✓ Extracted 801 pages

✂️  Chunking text (size=500, overlap=100)...
  ✓ Created 6431 chunks
  ✓ Avg length: 434 chars

🔢 Creating vector store...
  Encoding 6431 chunks...


Batches: 100%|██████████| 201/201 [03:53<00:00,  1.16s/it]

  Building FAISS index...
  ✓ Vector store ready (6431 vectors)

✅ KNOWLEDGE BASE READY
   Total chunks: 6431
   Embedding dim: 384


## Step 8: Retrieval Function

This finds the most relevant chunks for a given question.

In [8]:
def retrieve(query: str, top_k: int = 3) -> tuple:
    """
    Retrieve most relevant chunks for a query
    
    Args:
        query: Search query
        top_k: Number of chunks to retrieve
        
    Returns:
        Tuple of (relevant_chunks, similarities)
    """
    # Encode query
    query_embedding = embedder.encode([query])

    # Search
    distances, indices = index.search(
        query_embedding.astype('float32'),
        min(top_k, len(chunks))
    )

    # Convert distances to similarities
    similarities = 1 / (1 + distances[0])

    # Get chunks
    relevant_chunks = [chunks[i] for i in indices[0]]

    return relevant_chunks, similarities

print("✅ Function defined: retrieve()")

✅ Function defined: retrieve()


## Step 9: Answer Generation with Local Model

This uses the local HuggingFace model to generate answers based on retrieved context.

**Note**: Uses the global `llm` pipeline configured in Step 3.

In [ ]:
def generate_answer(query: str, context_chunks: List[str]) -> str:
    """
    Generate answer using local LLM
    
    Args:
        query: User's question
        context_chunks: Relevant context chunks
        
    Returns:
        Generated answer
    """
    # Combine context (keep it reasonable for the model)
    context_text = "\n".join(context_chunks)
    
    # Truncate if too long (FLAN-T5 has token limits)
    max_context_length = 1000
    if len(context_text) > max_context_length:
        context_text = context_text[:max_context_length] + "..."
    
    # Create prompt optimized for FLAN-T5
    prompt = f"""Answer the question based only on the context below.

Context: {context_text}

Question: {query}

Answer:"""
    
    try:
        # Generate answer using local model
        response = llm(prompt, max_length=150, do_sample=False)[0]['generated_text']
        return response.strip()
    except Exception as e:
        return f"Error generating answer: {str(e)}"

print("✅ Function defined: generate_answer()")

## Step 10: Complete RAG Pipeline

This combines retrieval + generation into one simple function.

In [10]:
def ask(question: str, top_k: int = 3) -> Dict:
    """
    Complete RAG pipeline: retrieve + generate
    
    Args:
        question: User's question
        top_k: Number of chunks to retrieve
        
    Returns:
        Dictionary with answer, sources, and similarities
    """
    if not index:
        return {
            "answer": "❌ Knowledge base not set up.",
            "sources": [],
            "similarities": []
        }

    # Retrieve relevant chunks
    relevant_chunks, similarities = retrieve(question, top_k)

    # Generate answer
    answer = generate_answer(question, relevant_chunks)

    return {
        "answer": answer,
        "sources": relevant_chunks,
        "similarities": similarities
    }

print("✅ Function defined: ask()")
print("\n🎉 RAG pipeline ready! Try asking questions in the next cell.")

✅ Function defined: ask()

🎉 RAG pipeline ready! Try asking questions in the next cell.


## Step 11: Test the RAG System!

Ask questions about your documents here.

In [11]:
# Test with sample questions
test_questions = [
    "What is regression analysis?",
    "What is econometrics?",
    "What are the main topics in this book?"
]

print("=" * 60)
print("TESTING RAG SYSTEM")
print("=" * 60)

for q in test_questions:
    print(f"\n❓ Q: {q}")
    result = ask(q, top_k=3)
    print(f"💬 A: {result['answer']}")
    print(f"📊 Similarities: {[f'{s:.2f}' for s in result['similarities']]}")
    print("-" * 60)

TESTING RAG SYSTEM

❓ Q: What is regression analysis?
💬 A: ❌ Invalid HuggingFace token. Please check your token.
📊 Similarities: ['0.54', '0.54', '0.53']
------------------------------------------------------------

❓ Q: What is econometrics?
💬 A: ❌ Invalid HuggingFace token. Please check your token.
📊 Similarities: ['0.76', '0.70', '0.65']
------------------------------------------------------------

❓ Q: What are the main topics in this book?
💬 A: ❌ Invalid HuggingFace token. Please check your token.
📊 Similarities: ['0.51', '0.51', '0.50']
------------------------------------------------------------


## Step 12: Interactive Q&A

Ask your own questions!

In [12]:
# Ask your own question here
my_question = "What is regression analysis?"  # Change this!

print(f"\n❓ Your Question: {my_question}\n")

result = ask(my_question, top_k=3)

print(f"💬 Answer:\n{result['answer']}\n")

print("📚 Source Chunks:")
for i, (source, sim) in enumerate(zip(result['sources'], result['similarities']), 1):
    print(f"\n{i}. (Similarity: {sim:.2f})")
    print(f"   {source[:200]}...")


❓ Your Question: What is regression analysis?

💬 Answer:
❌ Invalid HuggingFace token. Please check your token.

📚 Source Chunks:

1. (Similarity: 0.54)
   ceptual framework used in this text is the multiple regression model, the  mainstay of econometrics.This model, introduced in Part II, provides a mathematical way to quantify how a change in one varia...

2. (Similarity: 0.54)
   OLS algorithm. Regression software typically computes  the OLS fixed effects estimator in two steps.In the first step, the entity-specific average is subtracted from each variable. In the second step,...

3. (Similarity: 0.53)
   s explain how to use multiple regression to analyze the  relationship among variables in a data set.In this chapter, we step back and ask,  What makes a study that uses multiple regression reliable or...


## Step 13: Launch Gradio Interface (Optional)

Create a web interface for easier interaction.

In [ ]:
def create_gradio_interface():
    """
    Create Gradio web interface
    """
    def chatbot_response(message, history):
        result = ask(message, top_k=3)

        response = f"{result['answer']}\n\n"

        if result['sources']:
            response += "---\n**📚 Sources:**\n"
            for i, (source, sim) in enumerate(
                zip(result['sources'][:2], result['similarities'][:2]),
                1
            ):
                source_preview = source[:100].replace('\n', ' ')
                response += f"\n{i}. (Similarity: {sim:.2f}) {source_preview}...\n"

        return response

    interface = gr.ChatInterface(
        chatbot_response,
        title="📚 RAG Chatbot - 100% Local!",
        description=f"Ask questions about your PDF documents! Uses {MODEL_NAME} running locally - no API needed!",
        examples=[
            "What is this document about?",
            "Explain the main concept",
            "Summarize the key points",
        ],
        theme="soft",
    )

    return interface

# Launch the interface
print("🌐 Launching Gradio interface...")
print("   Press the stop button in Jupyter to stop the server.\n")

demo = create_gradio_interface()
demo.launch(share=False)  # Set share=True for a public link

## Summary

You've built a complete RAG system! Here's what you learned:

### Components
1. **PDF Processing** - Extract text from documents
2. **Text Chunking** - Split into manageable pieces with overlap
3. **Embeddings** - Create vector representations (local)
4. **FAISS Index** - Fast similarity search
5. **Retrieval** - Find relevant chunks
6. **Local LLM** - Generate answers using HuggingFace transformers

### Key Features
- ✅ **100% Free** - No API tokens or costs
- ✅ **100% Local** - Everything runs on your computer
- ✅ **Privacy** - Your documents never leave your machine
- ✅ **Production-ready** - Same patterns used in real applications
- ✅ **Works with HuggingFace Hub** - Uses transformers library

### Models Used
- **Embeddings**: `all-MiniLM-L6-v2` (sentence-transformers)
- **Generation**: `google/flan-t5-large` (can change to `flan-t5-base` for speed)

### Next Steps
- Try different questions
- Add more PDF documents
- Experiment with `top_k` parameter
- Try the Gradio interface
- Switch to `google/flan-t5-base` for faster responses (change in Step 1)

### Troubleshooting
- **Slow responses**: Try using `google/flan-t5-base` instead of `flan-t5-large`
- **Out of memory**: Reduce chunk size or use smaller model
- **No PDFs found**: Add PDFs to `documents/` folder
- **Model download issues**: Check internet connection - models download once and are cached